In [79]:
import pandas as pd
import requests
import os
import numpy as np
import json
import re

# 收集

In [2]:
# twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv','r', encoding = False)
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv', encoding = "iso-8859-1")
# twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv', sep=r'(?:,|\s+)', engine='python')

In [3]:
## 获取文件推特图片预测 image_predictions
### 1.确定image_predictions 的下载网址链接
image_predictions_url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv'
### 2.下载和保存文件
response = requests.get(image_predictions_url)
with open('image_predictions.csv', mode = 'wb') as file:
    file.write(response.content)
image_predictions = pd.read_csv('image_predictions.csv', sep = '\t')

In [4]:
## 获取文件3 每条推特的数据
tweet_json = pd.read_json('tweet_json.txt',lines = True)

# 评估

##### 质量
###### `WeRateDogs 推特档案(twitter_archive_enhanced)`的评估
- timestamp, retweeted_status_timestamp 应当调整为时间序列
- tweet_id 应当为字符串str
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id 采用 fillna 来将其转化为int
- rating_numerator分数最大值为1776，不合理
- rating_denominator分数最大值为170，不合理，这些评级通常以 10 作为分母
- tweet_id‘832215909146226688’的rating_denominator应当为9.75，而不是75
- 缺少了推特的网址链接 expanded_urls
- name为'a', 'an', 'the'都是原始数据中缺失值，应当替换replace为'None'
- 转发的推相当于对于某条狗的重复评价，应当删除该行数据。

###### `image_predictions` 的评估
- tweet_id 应当为字符串str
- 缺少图片推测（twitter中为2356个，图片推测仅为2075个）

###### `tweet_json` 的评估
- possibly_sensitive、possibly_sensitive_appealable 应当为布尔型bool
- 缺少api获取成果（twitter档案中为2356个，api获取成果仅为2352个）
- 有效性：in_reply_to_status_id、in_reply_to_user_id、retweeted_status_id、retweeted_status_user_id 应当为字符串str
- id和id_str 重复，删除str列
- possibly_sensitive和possibly_sensitive_appealable 重复，删除possibly_sensitive_appealable列
- extended_entities、possibly_sensitive 缺少数据


##### 整洁度
###### `WeRateDogs 推特档案(twitter_archive_enhanced)`的评估
- tweet_json中的转发数（retweet_count）和喜欢数（favorite_count）应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
- 神经网络推测出的狗的品种 (或其他物体、动物等），即可信度最高的image_predictions中的p1，应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
`twitter_archive_enhanced 的评估`

In [7]:
twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [57]:
twitter_archive_enhanced.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [74]:
# 验证是否有重复项,结果为没有
twitter_archive_enhanced['tweet_id'].duplicated().sum()

0

In [44]:
# 完整度
twitter_archive_enhanced[twitter_archive_enhanced['expanded_urls'].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
185,856330835276025856,NaN,NaN,2017-04-24 02:15:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Jenna_Marbles: @dog_rates Thanks for ratin...,8.563302e+17,66699013.0,2017-04-24 02:13:14 +0000,NaN,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None


In [6]:
# 查找出 转发的tweet，结果为有，共181条转发推。
twitter_archive_enhanced[twitter_archive_enhanced['retweeted_status_id'].notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 19 to 2260
Data columns (total 17 columns):
tweet_id                      181 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     181 non-null object
source                        181 non-null object
text                          181 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 180 non-null object
rating_numerator              181 non-null int64
rating_denominator            181 non-null int64
name                          181 non-null object
doggo                         181 non-null object
floofer                       181 non-null object
pupper                        181 non-null object
puppo                         181 non-null object
dtypes: float64(4), int64(3), object(10)
memor

In [87]:
# 找出狗名字的重复数据
twitter_archive_enhanced[twitter_archive_enhanced['name'].duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
12,889665388333682689,NaN,NaN,2017-07-25 01:55:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo that seems to be on the fence a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889665388...,13,10,None,None,None,None,puppo
23,887473957103951883,NaN,NaN,2017-07-19 00:47:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Canela. She attempted some fancy porch...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
24,887343217045368832,NaN,NaN,2017-07-18 16:08:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",You may not have known you needed to see this ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887343217...,13,10,None,None,None,None,None
25,887101392804085760,NaN,NaN,2017-07-18 00:07:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This... is a Jubilant Antarctic House Bear. We...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887101392...,12,10,None,None,None,None,None
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,19607400.0,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
35,885518971528720385,NaN,NaN,2017-07-13 15:19:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I have a new hero and his name is Howard. 14/1...,NaN,NaN,NaN,https://twitter.com/4bonds2carbon/status/88551...,14,10,None,None,None,None,None
37,885167619883638784,NaN,NaN,2017-07-12 16:03:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a corgi undercover as a malamute....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/885167619...,13,10,None,None,None,None,None
41,884441805382717440,NaN,NaN,2017-07-10 15:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I present to you, Pup in Hat. Pup in Hat is gr...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/884441805...,14,10,None,None,None,None,None


In [93]:
# name为'a', 'an', 'the'都是原始数据中缺失值，因为并没有狗的名字会叫做冠词。
twitter_archive_enhanced['name'].value_counts()

None           745
a               55
Charlie         12
Lucy            11
Oliver          11
Cooper          11
Penny           10
Lola            10
Tucker          10
Winston          9
Bo               9
the              8
Sadie            8
Daisy            7
Bailey           7
Buddy            7
Toby             7
an               7
Bella            6
Rusty            6
Leo              6
Koda             6
Scout            6
Jack             6
Jax              6
Milo             6
Oscar            6
Stanley          6
Dave             6
very             5
              ... 
Pherb            1
Lilli            1
Mark             1
Ricky            1
Toffee           1
Harlso           1
Vinnie           1
Sunshine         1
Vinscent         1
Rodney           1
Nida             1
Tassy            1
Tater            1
this             1
Coleman          1
Kobe             1
Oddie            1
Clarq            1
Binky            1
Einstein         1
Sobe             1
Edmund      

In [162]:
# rating_numerator的统计
twitter_archive_enhanced['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [163]:
# rating_denominator的统计
# 评级通常以 10 作为分母，所以结果中不是10的均应当
twitter_archive_enhanced['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [180]:
# 不用
# 找到评价基准分数大于10的，存入test_0227
# twitter_archive_enhanced_clean
# test_0227 = twitter_archive_enhanced_clean[(np.abs(twitter_archive_enhanced_clean['rating_denominator']) > 10).any(1)]
### test_0227 = twitter_archive_enhanced_clean[(np.abs(twitter_archive_enhanced_clean.rating_denominator) <= 10)]

In [182]:
# 不用
### test_0227['rating_numerator'].value_counts()

12      558
11      463
10      461
13      351
9       156
8       102
14       54
7        54
5        37
6        32
3        19
4        15
2         9
1         9
75        2
0         2
420       2
15        2
26        1
24        1
182       1
27        1
960       1
1776      1
17        1
666       1
Name: rating_numerator, dtype: int64

`image_predictions 的评估`

In [49]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [56]:
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [84]:
# 每条推的照片数量，结果发现最多4张，最少1张
image_predictions['img_num'].sort_values()

In [73]:
# 验证是否有重复项，结果为没有重复
image_predictions['tweet_id'].duplicated().sum()

0

`tweet_json 的评估`

In [51]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2352 entries, 0 to 2351
Data columns (total 31 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2352 non-null datetime64[ns]
display_text_range               2352 non-null object
entities                         2352 non-null object
extended_entities                2073 non-null object
favorite_count                   2352 non-null int64
favorited                        2352 non-null bool
full_text                        2352 non-null object
geo                              0 non-null float64
id                               2352 non-null int64
id_str                           2352 non-null int64
in_reply_to_screen_name          78 non-null object
in_reply_to_status_id            78 non-null float64
in_reply_to_status_id_str        78 non-null float64
in_reply_to_user_id              78 non-null float64
in_reply_to_user_id_str          78 n

In [5]:
# 从tweet_json中提取三个关键列'id','retweet_count','favorite_count',根据这三个指标建立tweet_keys
tweet_info_keys = ['id','retweet_count','favorite_count']
tweet_keys = pd.DataFrame(tweet_json, columns=tweet_info_keys)

In [14]:
tweet_keys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 3 columns):
id                2352 non-null int64
retweet_count     2352 non-null int64
favorite_count    2352 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [69]:
tweet_keys.sample(10)

,id,retweet_count,favorite_count
1320,706169069255446529,2497,4269
44,883482846933004288,10336,46741
1357,703079050210877440,3480,8044
1549,688908934925697024,869,2304
1693,681231109724700672,539,2616
1452,695314793360662529,1664,3991
492,813172488309972993,2227,10344
2337,666094000022159362,78,168
1748,678969228704284672,527,1788
1530,689993469801164801,516,1610


In [70]:
tweet_keys.describe()

,id,retweet_count,favorite_count
count,2.352000e+03,2352.000000,2352.000000
mean,7.425913e+17,3134.932398,8109.198980
std,6.846210e+16,5237.846296,11980.795669
min,6.660209e+17,0.000000,0.000000
25%,6.783949e+17,618.000000,1417.000000
50%,7.193536e+17,1456.500000,3596.500000
75%,7.991219e+17,3628.750000,10118.000000
max,8.924206e+17,79116.000000,132318.000000


In [72]:
# 验证是否有重复项，结果为没有重复
tweet_keys['id'].duplicated().sum()

0

In [59]:
tweet_json.describe()

,contributors,coordinates,favorite_count,geo,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,possibly_sensitive,possibly_sensitive_appealable,quoted_status_id,quoted_status_id_str,retweet_count
count,0.0,0.0,2352.000000,0.0,2.352000e+03,2.352000e+03,7.800000e+01,7.800000e+01,7.800000e+01,7.800000e+01,2211.0,2211.0,2.900000e+01,2.900000e+01,2352.000000
mean,NaN,NaN,8109.198980,NaN,7.425913e+17,7.425913e+17,7.455079e+17,7.455079e+17,2.014171e+16,2.014171e+16,0.0,0.0,8.162686e+17,8.162686e+17,3134.932398
std,NaN,NaN,11980.795669,NaN,6.846210e+16,6.846210e+16,7.582492e+16,7.582492e+16,1.252797e+17,1.252797e+17,0.0,0.0,6.164161e+16,6.164161e+16,5237.846296
min,NaN,NaN,0.000000,NaN,6.660209e+17,6.660209e+17,6.658147e+17,6.658147e+17,1.185634e+07,1.185634e+07,0.0,0.0,6.721083e+17,6.721083e+17,0.000000
25%,NaN,NaN,1417.000000,NaN,6.783949e+17,6.783949e+17,6.757419e+17,6.757419e+17,3.086374e+08,3.086374e+08,0.0,0.0,7.888183e+17,7.888183e+17,618.000000
50%,NaN,NaN,3596.500000,NaN,7.193536e+17,7.193536e+17,7.038708e+17,7.038708e+17,4.196984e+09,4.196984e+09,0.0,0.0,8.340867e+17,8.340867e+17,1456.500000
75%,NaN,NaN,10118.000000,NaN,7.991219e+17,7.991219e+17,8.257804e+17,8.257804e+17,4.196984e+09,4.196984e+09,0.0,0.0,8.664587e+17,8.664587e+17,3628.750000
max,NaN,NaN,132318.000000,NaN,8.924206e+17,8.924206e+17,8.862664e+17,8.862664e+17,8.405479e+17,8.405479e+17,0.0,0.0,8.860534e+17,8.860534e+17,79116.000000


In [161]:
# 查看tweet.json数据
tweet_json.sample(5)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,possibly_sensitive_appealable,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,truncated,user
1250,NaN,NaN,2016-03-18 02:46:49,"[0, 108]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 710658682619617280, 'id_str'...",2513,False,Here's a brigade of puppers. All look very pre...,NaN,...,0.0,NaN,NaN,NaN,633,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
599,NaN,NaN,2016-11-15 21:49:12,"[0, 103]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 670444949847023616, 'id_str'...",0,False,RT @dog_rates: This is Paull. He just stubbed ...,NaN,...,0.0,NaN,NaN,NaN,2142,False,{'created_at': 'Sat Nov 28 03:31:48 +0000 2015...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1982,NaN,NaN,2015-12-04 21:05:23,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 672884420823527424, 'id_str'...",1653,False,Marvelous dog here. Rad ears. Not very soft. L...,NaN,...,0.0,NaN,NaN,NaN,895,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2350,NaN,NaN,2015-11-15 23:05:30,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666029276303482880, 'id_str'...",132,False,This is a western brown Mitsubishi terrier. Up...,NaN,...,0.0,NaN,NaN,NaN,48,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2226,NaN,NaN,2015-11-22 01:20:44,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 668237637640183808, 'id_str'...",6573,False,This is Torque. He served his nickel. Better n...,NaN,...,0.0,NaN,NaN,NaN,3083,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


In [110]:
# 完整度
tweet_json[tweet_json['extended_entities'].isnull()]

In [11]:
# tweet_json['truncated'].duplicated()
# tweet_json['truncated'].duplicated().sum()
tweet_json['user']

# 清理

### 保存原始数据的副本

In [144]:
twitter_archive_enhanced_clean = twitter_archive_enhanced.copy()
image_predictions_clean = image_predictions.copy()
tweet_json_clean = tweet_json.copy()
tweet_keys_clean = tweet_keys.copy()

### 整洁度
#### tweet_json中的转发数（retweet_count）和喜欢数（favorite_count）应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
##### 定义
将tweet_json中的retweet_count 和favorite_count导入一个 DataFrame 中，并它通过tweet_id拼接到 twitter_archive_enhanced_clean中。

##### 代码

In [145]:
# tweet_keys_clean中的id列改名为tweet_id后，twitter_archive_enhanced_clean就可以和tweet_keys_clean拼接。
tweet_keys_clean.rename(columns={'id':'tweet_id'}, inplace = True)

# 拼接
twitter_archive_enhanced_clean = pd.merge(twitter_archive_enhanced_clean, tweet_keys_clean, on='tweet_id', how='left')

##### 测试

In [97]:
tweet_keys_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2352 entries, 0 to 2351
Data columns (total 3 columns):
tweet_id          2352 non-null int64
retweet_count     2352 non-null int64
favorite_count    2352 non-null int64
dtypes: int64(3)
memory usage: 73.5 KB


In [52]:
twitter_archive_enhanced_clean[twitter_archive_enhanced_clean['retweet_count'].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None,NaN,NaN
118,869988702071779329,NaN,NaN,2017-05-31 18:47:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: We only rate dogs. This is quit...,8.591970e+17,4.196984e+09,2017-05-02 00:04:57 +0000,https://twitter.com/dog_rates/status/859196978...,12,10,quite,None,None,None,None,NaN,NaN
155,861769973181624320,NaN,NaN,2017-05-09 02:29:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: ""Good afternoon class today we'...",8.066291e+17,4.196984e+09,2016-12-07 22:38:52 +0000,https://twitter.com/dog_rates/status/806629075...,13,10,None,None,None,None,None,NaN,NaN
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None,NaN,NaN


In [53]:
# 可发现retweet_count 和favorite_count的index已经加入twitter_archive_enhanced_clean中
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count                 23

#### image_predictions中的p1，应当为WeRateDogs 推特档案(twitter_archive_enhanced)的一部分
##### 定义
通过 image_predictions 中的tweet_id，将其中的p1拼接到twitter_archive_enhanced_clean中。
##### 代码

In [146]:
twitter_archive_enhanced_clean = pd.merge(twitter_archive_enhanced_clean, image_predictions, on='tweet_id', how='left')

##### 测试

In [99]:
# 可发现p1,p1_conf和p1_dog已经加入twitter_archive_enhanced_clean中
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count                 23

### 质量
#### `twitter_archive_enhanced`中name为'a', 'an', 'the'都是原始数据中缺失值，替换为'None'
##### 定义
使用函数，将 a, an, the 转变为 None。

##### 代码

In [147]:
# name的映射
a_clean = {'a': 'None', 'an': 'None', 'the': 'None'}
# 替换名字函数
def replace_name(twitter_archive_enhanced):
    if twitter_archive_enhanced['name'] in a_clean.keys():
        clean_name = a_clean[twitter_archive_enhanced['name']]
        return clean_name
    else:
        return twitter_archive_enhanced['name']

twitter_archive_enhanced_clean['name'] = twitter_archive_enhanced_clean.apply(replace_name, axis=1)

##### 测试

In [57]:
# 'name'为'a', 'an', 'the'均已经替换为'None'
twitter_archive_enhanced_clean['name'].value_counts()

None            815
Charlie          12
Cooper           11
Oliver           11
Lucy             11
Penny            10
Lola             10
Tucker           10
Bo                9
Winston           9
Sadie             8
Toby              7
Daisy             7
Bailey            7
Buddy             7
Scout             6
Milo              6
Stanley           6
Dave              6
Bella             6
Jax               6
Koda              6
Rusty             6
Leo               6
Jack              6
Oscar             6
Oakley            5
Sammy             5
Bentley           5
Louis             5
               ... 
Ralphie           1
Trigger           1
Ozzie             1
Gustaf            1
Noosh             1
Iroh              1
Rhino             1
Dale              1
Tuck              1
Lilli             1
Newt              1
Remus             1
Banditt           1
Zeek              1
Birf              1
Alexanderson      1
Thor              1
Hanz              1
Lenox             1


#### 错误的数据类型
##### 定义
- timestamp, retweeted_status_timestamp 中均有‘ +0000’ 的后缀，应当删除，并转化为 datetime 数据类型。
- tweet_id 应当为字符串str
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id 采用 fillna 来将其转化为int

##### 代码

In [148]:
# 删除timestamp的‘ +0000’ 的后缀
twitter_archive_enhanced_clean.timestamp = twitter_archive_enhanced_clean.timestamp.str.strip('\s\+0000')
twitter_archive_enhanced_clean.retweeted_status_timestamp = twitter_archive_enhanced_clean.retweeted_status_timestamp.str.strip('\s\+0000')
# 将timestamp转化为 datetime 类型
twitter_archive_enhanced_clean.timestamp = pd.to_datetime(twitter_archive_enhanced_clean.timestamp)
twitter_archive_enhanced_clean.retweeted_status_timestamp = pd.to_datetime(twitter_archive_enhanced_clean.retweeted_status_timestamp)

In [149]:
# 将tweet_id转为str数据类型
twitter_archive_enhanced_clean.tweet_id = twitter_archive_enhanced_clean.tweet_id.astype('str')
# 回复相关id和转发相关id为float64，且存在NaN，则采用 fillna 来将其转化为int
# 参考 https://stackoverflow.com/questions/47013973/pandas-converts-float64-to-int
twitter_archive_enhanced_clean.in_reply_to_status_id = twitter_archive_enhanced_clean.in_reply_to_status_id.fillna(0).astype(np.int64)
twitter_archive_enhanced_clean.in_reply_to_user_id = twitter_archive_enhanced_clean.in_reply_to_user_id.fillna(0).astype(np.int64)
twitter_archive_enhanced_clean.retweeted_status_id = twitter_archive_enhanced_clean.retweeted_status_id.fillna(0).astype(np.int64)
twitter_archive_enhanced_clean.retweeted_status_user_id = twitter_archive_enhanced_clean.retweeted_status_user_id.fillna(0).astype(np.int64)

##### 测试

In [19]:
# twitter_archive_enhanced_clean.retweeted_status_id

In [60]:
# 测试timestamp, retweeted_status_timestamp 中‘ +0000’ 的后缀是否已经删除，并转化为 datetime 数据类型。
twitter_archive_enhanced_clean.sample(30)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1813,676776431406465024,0,0,2015-12-15 14:50:49,"<a href=""http://twitter.com/download/iphone"" r...","When someone yells ""cops!"" at a party and you ...",0,0,NaT,https://twitter.com/dog_rates/status/676776431...,...,1.0,doormat,0.201346,False,dishwasher,0.191749,False,microwave,0.038110,False
2101,670691627984359425,0,0,2015-11-28 19:51:59,"<a href=""http://twitter.com/download/iphone"" r...",This is Ester. He has a cocaine problem. This ...,0,0,NaT,https://twitter.com/dog_rates/status/670691627...,...,1.0,Shetland_sheepdog,0.071124,True,home_theater,0.068398,False,American_Staffordshire_terrier,0.066964,True
1717,680191257256136705,0,0,2015-12-25 01:00:07,"<a href=""http://twitter.com/download/iphone"" r...",Here's a sleepy Christmas pupper 11/10 https:/...,0,0,NaT,https://twitter.com/dog_rates/status/680191257...,...,1.0,Brittany_spaniel,0.733253,True,Welsh_springer_spaniel,0.251634,True,English_springer,0.009243,True
2195,668872652652679168,0,0,2015-11-23 19:24:02,"<a href=""http://twitter.com/download/iphone"" r...",This is AmÃ©lie. She is a confident white coll...,0,0,NaT,https://twitter.com/dog_rates/status/668872652...,...,1.0,teddy,0.413379,False,pillow,0.325623,False,miniature_schnauzer,0.035537,True
477,815990720817401858,0,0,2017-01-02 18:38:42,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jack. He's one of the rare doggos that do...,0,0,NaT,https://www.gofundme.com/surgeryforjacktheminp...,...,1.0,Chihuahua,0.428756,True,miniature_pinscher,0.103912,True,Staffordshire_bullterrier,0.088959,True
2080,670823764196741120,0,0,2015-11-29 04:37:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Remington. He's a man dime. 12/10 http...,0,0,NaT,https://twitter.com/dog_rates/status/670823764...,...,1.0,Labrador_retriever,0.947453,True,German_short-haired_pointer,0.017001,True,Weimaraner,0.015432,True
1014,747204161125646336,0,0,2016-06-26 23:05:29,"<a href=""http://twitter.com/download/iphone"" r...",This is Clark. He's deadly af. Clearly part sh...,0,0,NaT,https://twitter.com/dog_rates/status/747204161...,...,2.0,coil,0.533699,False,dugong,0.087959,False,rain_barrel,0.039221,False
183,856543823941562368,0,0,2017-04-24 16:22:16,"<a href=""http://twitter.com/download/iphone"" r...",This is Callie. She'll be your navigator today...,0,0,NaT,https://twitter.com/dog_rates/status/856543823...,...,1.0,Boston_bull,0.306910,True,Siamese_cat,0.191218,False,Chihuahua,0.189288,True
2271,667495797102141441,0,0,2015-11-20 00:12:54,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Philippe from Soviet Russia. Commandin...,0,0,NaT,https://twitter.com/dog_rates/status/667495797...,...,1.0,Chihuahua,0.143957,True,Christmas_stocking,0.118651,False,ski_mask,0.092482,False
2134,670069087419133954,0,0,2015-11-27 02:38:14,"<a href=""http://twitter.com/download/iphone"" r...",This is Randall. He's from Chernobyl. Built pl...,0,0,NaT,https://twitter.com/dog_rates/status/670069087...,...,1.0,boathouse,0.313829,False,birdhouse,0.138331,False,ashcan,0.045673,False


In [61]:
# 查看 数据类型是否转化为 datetime，str 和 int 类型
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count    

#### 转发的推相当于对于该狗的重复评价，应当删除该行狗的数据。
##### 定义
- 转发的tweet应当删除，找出带有retweeted_status_id的行，然后去除该行
##### 代码

In [150]:
# retweeted_status_id为0的推文是没有转发的。
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean[np.abs(twitter_archive_enhanced_clean['retweeted_status_id']) == 0]

##### 测试

In [63]:
# retweeted_status_id，retweeted_status_user_id，retweeted_status_timestamp这三列的值已经均为NaN。
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 30 columns):
tweet_id                      2175 non-null object
in_reply_to_status_id         2175 non-null int64
in_reply_to_user_id           2175 non-null int64
timestamp                     2175 non-null datetime64[ns]
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           2175 non-null int64
retweeted_status_user_id      2175 non-null int64
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null int64
rating_denominator            2175 non-null int64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
retweet_count      

In [64]:
# 原来19行 tweet_id为888202515573088257的推文，是转发推文，已经被去除了。
twitter_archive_enhanced_clean.head(30)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,0,0,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,0,0,NaT,https://twitter.com/dog_rates/status/892420643...,...,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,0,0,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,0,0,NaT,https://twitter.com/dog_rates/status/892177421...,...,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,0,0,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,0,0,NaT,https://twitter.com/dog_rates/status/891815181...,...,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,0,0,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,0,0,NaT,https://twitter.com/dog_rates/status/891689557...,...,1.0,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,0,0,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,0,0,NaT,https://twitter.com/dog_rates/status/891327558...,...,2.0,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
5,891087950875897856,0,0,2017-07-29 00:08:17,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,0,0,NaT,https://twitter.com/dog_rates/status/891087950...,...,1.0,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
6,890971913173991426,0,0,2017-07-28 16:27:12,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,0,0,NaT,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",...,1.0,Appenzeller,0.341703,True,Border_collie,0.199287,True,ice_lolly,0.193548,False
7,890729181411237888,0,0,2017-07-28 00:22:40,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,0,0,NaT,https://twitter.com/dog_rates/status/890729181...,...,2.0,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True
8,890609185150312448,0,0,2017-07-27 16:25:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,0,0,NaT,https://twitter.com/dog_rates/status/890609185...,...,1.0,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True
9,890240255349198849,0,0,2017-07-26 15:59:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,0,0,NaT,https://twitter.com/dog_rates/status/890240255...,...,1.0,Pembroke,0.511319,True,Cardigan,0.451038,True,Chihuahua,0.029248,True


#### 原始数据不合理
##### 定义
- rating_numerator分数最大值为1776，不合理。rating_denominator分数最大值为170，不合理，这些评级通常以 10 作为分母。
- 针对以上两个问题确定了新指标RATING，即每条狗的rating_numerator/rating_denominator的比值，并去除NaN值和非常大的奇异值（本项目认为大于2即为奇异值），RATING的高低可以有效体现每条狗的评级，分数越大，表明狗评级越高、越收到欢迎。之后的分析结论也是基于RATING一列。

##### 代码

In [40]:
# rating_numerator_morethan20 = twitter_archive_enhanced_clean[(np.abs(twitter_archive_enhanced_clean.rating_numerator) >= 20)]
# rating_numerator_morethan20

In [151]:
# 定义 RATING
twitter_archive_enhanced_clean['RATING'] = twitter_archive_enhanced_clean['rating_numerator']/twitter_archive_enhanced_clean['rating_denominator']
# 去除 twitter_archive_enhanced_clean['RATING'] 中的NaN值和非常大的奇异值（本项目认为大于20即为奇异值）
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean[np.abs(twitter_archive_enhanced_clean['RATING']) <= 2]

##### 测试

In [95]:
twitter_archive_enhanced_clean.sample(30)

In [70]:
# 查看是否还存在奇异值和NaN。
twitter_archive_enhanced_clean['RATING'].value_counts()

1.200000    504
1.000000    444
1.100000    431
1.300000    307
0.900000    156
0.800000     98
0.700000     53
1.400000     43
0.500000     37
0.600000     32
0.300000     19
0.400000     15
0.200000     11
0.100000      7
0.000000      2
0.818182      1
1.700000      1
0.733333      1
0.636364      1
1.250000      1
1.500000      1
Name: RATING, dtype: int64

In [152]:
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 31 columns):
tweet_id                      2175 non-null object
in_reply_to_status_id         2175 non-null int64
in_reply_to_user_id           2175 non-null int64
timestamp                     2175 non-null datetime64[ns]
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           2175 non-null int64
retweeted_status_user_id      2175 non-null int64
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null int64
rating_denominator            2175 non-null int64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
retweet_count      

# 分析和结论

### 评级最高的Top_3 狗的品种

找出评级（RATING）前三的狗的品种（p1）

#### 代码

In [173]:
# p1_dog为True的推文中提到的才是狗。所以建立DataFrame twitter_archive_enhanced_clean_p1，整理出为狗的数据。
# 首先将 .p1_dog转化为bool值
twitter_archive_enhanced_clean.p1_dog = twitter_archive_enhanced_clean.p1_dog.astype(np.bool)
# twitter_archive_enhanced_clean_p1就是推文图片中是狗的数据
twitter_archive_enhanced_clean_p1 = twitter_archive_enhanced_clean[twitter_archive_enhanced_clean.p1_dog == True]
# p1_Rating为每种狗的评级均值
p1_Rating = twitter_archive_enhanced_clean_p1['RATING'].groupby(twitter_archive_enhanced_clean_p1['p1']).mean()
p1_Rating